# Prospects for high spectral and spatial resolution with SKA1-mid #

SKA1-mid will revolutionize resolve 21-cm HI surveys. Current radio interfometers limit high brightness sensitivity HI observations to $\sim6^{''}$. SKA1-mid will enable $\sim1^{''}$ observations that, currently, we can only achieve within the Local Group (approx. $D<1$ Mpc).

This notebook provides rough estimates of the time required to reproduce our current Local Group HI VLA observations for more ditance galaxies with SKA1-mid. Our initial M33 VLA observations are presented in [Koch et al. 2018](https://ui.adsabs.harvard.edu/abs/2018MNRAS.479.2505K/abstract) and the M31 observations will be published in a submitted paper (Koch et al. 2021).

I have scaled the sensitivity estimates from the expectations provided in Table 7 of [Braun et al. 2019](https://ui.adsabs.harvard.edu/abs/2019arXiv191212699B/abstract). *These estimates should be treated as approximate and do not represent a rigorous calculation.*

In [ ]:
%matplotlib inline

In [ ]:
import numpy as np
import astropy.units as u

import matplotlib.pyplot as plt



From Table 7, the 1 hr sensitivity at ~1.4 GHz is:
$\sigma_{line} = 186$ uJy/beam.

The channel width is assumed to be $\delta \nu / \nu_c=10^{-4}$, where $\nu_c=1.43$. The channel width is then:

In [ ]:
nu_HI = 1420405751.7667 * u.Hz
nu_c = 1.43 * u.GHz


chanwidth_freq = (1e-4 * nu_c).to(u.kHz)

chanwidth_freq

That corresponds to a velocity width of:

In [ ]:
chanwidth_vel = nu_c.to(u.km / u.s, u.doppler_radio(nu_HI)) - (nu_c + chanwidth_freq).to(u.km / u.s, u.doppler_radio(nu_HI))

chanwidth_vel

This is a particularly coarse spectral width. For comparison with the Local Group VLA observations, we want to scale the senstivity down to $<1$ km/s channels.

For 1 hr of observations, Table 7 gives the line sensitivity as:

In [ ]:
sigma_line = 1.86 * u.uJy / u.beam

This sensitivity is valid for beam sizes from 0.587 to 78.3 arcsec.

To look at the range of surface brightness sensitivities, we will define the extreme beam sizes:

In [ ]:
from radio_beam import Beam

min_beam = Beam(0.587 * u.arcsec)
max_beam = Beam(78.3 * u.arcsec)

The corresponding surface brightness (SB) sensitivities then range from:

In [ ]:
sb_min_beam_rms = sigma_line.to(u.K, u.brightness_temperature(nu_HI, min_beam.sr))
sb_max_beam_rms = sigma_line.to(u.K, u.brightness_temperature(nu_HI, max_beam.sr))

print(f"Min beam surface brightness rms: {sb_min_beam_rms}")
print(f"Max beam surface brightness rms: {sb_max_beam_rms}")

The minimum velocity channel width for resolving an HI spectra with a 10 km/s line width is:

In [ ]:
resolved_velwidth_min = 10 * u.km / u.s * 2.35 / 2.

resolved_velwidth_min

This assumes [the Koch et al. 2018b](https://ui.adsabs.harvard.edu/abs/2018RNAAS...2..220K/abstract) result for the minimum channel sampling needed to avoid line broadening due to finite channel widths:

$\sigma_{\rm min res.} \approx {\rm FWHM} / 2$

The surface brightness sensitivities from above with this channel width are:

In [ ]:
chanwidth_ratio = (chanwidth_vel / resolved_velwidth_min).value

sb_min_beam_rms_resolved = sb_min_beam_rms * chanwidth_ratio**0.5
sb_max_beam_rms_resolved = sb_max_beam_rms * chanwidth_ratio**0.5

print(f"Min beam surface brightness rms: {sb_min_beam_rms_resolved} with {resolved_velwidth_min} channels")
print(f"Max beam surface brightness rms: {sb_max_beam_rms_resolved} with {resolved_velwidth_min} channels")

The minimum beam is immediately promising since high-res spatially-resolved HI should peak at >60 K.

What about at the opposite end of very resolved HI spectra, similar to the Local Group observations?

We will choose our finest spectral resolution of 0.42 km/s, where the thermal line width of 120 K atomic gas is resolved:

In [ ]:
resolved_velwidth_max =  0.42 * u.km / u.s

resolved_velwidth_max

In [ ]:
chanwidth_ratio_max = (chanwidth_vel / resolved_velwidth_max).value

print(f"Channel width ratio {chanwidth_ratio_max}")

sb_min_beam_rms_resolved_max = sb_min_beam_rms * chanwidth_ratio_max**0.5
sb_max_beam_rms_resolved_max = sb_max_beam_rms * chanwidth_ratio_max**0.5

print(f"Min beam surface brightness rms: {sb_min_beam_rms_resolved_max} with {resolved_velwidth_max} channels")
print(f"Max beam surface brightness rms: {sb_max_beam_rms_resolved_max} with {resolved_velwidth_max} channels")

This is very promising since we expect resolved HI to have a peak temperature of ~120 K based on M31 VLA observations on ~30 pc scales, and LMC ATCA at ~15 pc.

High spatial and spectral resolutions remains a challenge, even for SKA1-mid. Future HI spectral resolution observations will need to balance between these extremes to enable mapping nearby galaxies similar to current Local Group observations.

How much time would equivalent Local Group HI observations take with the SKA1-mid. Below is a function to roughly estimate the on-source time given a target beam size, sensitivity, and channel width.

In [ ]:

def hi_time_on_source_ska1mid(beamwidth, sb_rms, channel=0.42 * u.km / u.s):
    '''
    Estimate the integration time for SKA1-mid to reach the target SB sensitivity at the given
    beam size. Based on sensitivity estimates from Table 7 of Braun+2019.

    Parameters
    ----------
    beamwidth : `~astropy.units.Quantity`
        Beam FWHM, assuming a circular beam.
    sb_rms : `~astropy.units.Quantity`
        Target surface brightness rms senstivity.
    channel : ~astropy.units.Quantity`
        Spectral channel width in velocity.

    Returns
    -------
    integ_time : ~astropy.units.Quantity`
        Required on-source time with SKA1-mid.
    '''

    chanwidth_ratio = (chanwidth_vel / channel).value

    beam = Beam(beamwidth)

    if beam.sr < min_beam.sr or beam.sr > max_beam.sr:
        raise ValueError("The chosen beam size falls outside of the beam area range"
                         " for the noise estimates to be valid.")

    # Convert the SKA1-mid's sensitivity to K at the requested channel width
    ska_1hr_rms = sigma_line.to(u.K, u.brightness_temperature(nu_HI, beam.sr)) * chanwidth_ratio**0.5

    # print(ska_1hr_rms)
    # print(sb_rms / ska_1hr_rms)

    integ_time = (ska_1hr_rms / sb_rms)**2. * 1 * u.hr

    return integ_time


A nice even beam size is 1", which is well-matched to CO(2-1) mapping with Band 6 ALMA that many on-going surveys target.

For $10\sigma$ peak detections of a 100 K HI peak temperature, the integration time need with SKA1-mid is:

In [ ]:
hi_time_on_source_ska1mid(1. * u.arcsec, 10. * u.K, channel=0.42 * u.km / u.s)

SKA1-mid enables HI mapping at high spectral resolution of nearby galaxies in 1 hr.

To minimally resolve the H$_2$ disc scale height, we require resolving $\sim100$ pc scales. Keeping the linear resolution fixed, the beam size we need for a galaxy at $D=25$ Mpc is:

In [ ]:
lin_scale = 100 * u.pc
distance = 25 * u.Mpc

ang_res = ((lin_scale / distance) * u.rad).to(u.arcsec)

ang_res

We will target the equiv. sensitivity of our current M33 observations. Using the VLA's C configuration, the linear resolution is 80 pc for a sensitivity of 2.7 K per 0.42 km/s channel. The sensitivity smoothed to 100 pc is:

In [ ]:
m33_100pc_sb_rms = 2.7 * u.K * (80. * u.pc / (100. * u.pc))**2.

m33_100pc_sb_rms

This corresponds to a columnd density per channel of:

In [ ]:
(2.7 * u.K * 0.42 * u.km / u.s) * 1.82e18 * (u.cm**-2 / (u.K * u.km / u.s))

To reach this sensitivity for a target at 25 Mpc resolving 100 pc scales, SKA1-mid would require an integration time of:

In [ ]:
hi_time_on_source_ska1mid(ang_res, 1.7 * u.K, channel=0.42 * u.km / u.s)


Individual galaxies at this extreme distance range require a fair bit of integration time, but such studies will actually be _achievable_ for the firs time!

SKA1-mid can map 100 pc scales of the atomic ISM to distances of 25 Mpc. This capability will enable the first resolved surveys of HI for a galaxies that samples the star-forming main sequence (SFMS).

To demonstrate this, we will show the nearby galaxy sample observable by SKA1-mid out to 25 Mpc using the nearby galaxy sample from z0mgs ([Leroy et al. 2019](https://ui.adsabs.harvard.edu/abs/2019ApJS..244...24L/abstract)), a WISE+GALEX-selected sample of nearby galaxies.

We will read in the machine-readable sample table from the publication:

In [ ]:
from astropy.table import Table

tab = Table.read("apjsab3925t4_mrt.txt", format='ascii.cds')

In [ ]:
# Turn on saving plots

# import os

# if not os.path.exists('z0mgs_sample_figures'):
#     os.mkdir('z0mgs_sample_figures')

In [ ]:
# Set plotting theme with seaborn

import seaborn as sb

sb.set_theme(context='talk', palette='colorblind', style='ticks')

To show the galaxy population, we will use KDE density contours and place samples on top to demonstrate which samples trace the range of galaxy populations.

In all of the following plots, we show the specific star formation rate (SFR/$M_\star$) versus the stellar mass ($M_\star$). For restricted samples, I show the population with $M_\star > 10^8$ solar masses where the catalogue is more complete at larger distances.

In [ ]:

plt.figure(figsize=(8., 8.))

# plt.scatter(tab['logM*'], tab['logSFR'] - tab['logM*'])
sb.kdeplot(tab['logM*'], tab['logSFR'] - tab['logM*'])

plt.xlabel("log $M_\star$ ($M_\odot$)", fontsize=20)
plt.ylabel("log SFR / $M_\star$ (yr$^{-1}$)", fontsize=20)

plt.grid(True)

plt.xlim([7, 12])
plt.ylim([-14, -8])

# plt.savefig("z0mgs_sample_figures/sfrms_all.png", dpi=600, bbox_inches='tight')
# plt.savefig("z0mgs_sample_figures/sfrms_all.pdf", dpi=600, bbox_inches='tight')


M31 on this diagram is in the "green valley" and M33 falls close to the SFMS.

In [ ]:

plt.figure(figsize=(8., 8.))

sb.kdeplot(tab['logM*'], tab['logSFR'] - tab['logM*'])

# M31
plt.scatter(tab['logM*'][tab['D'] < 1][0],
            (tab['logSFR'] - tab['logM*'])[tab['D'] < 1][0],
            color=sb.color_palette('colorblind')[2],
            marker='s', s=160, zorder=10)
# M33
plt.scatter(tab['logM*'][tab['D'] < 1][2],
            (tab['logSFR'] - tab['logM*'])[tab['D'] < 1][2],
            color=sb.color_palette('colorblind')[2],
            marker='s', s=160, zorder=10)


plt.xlabel("log $M_\star$ ($M_\odot$)", fontsize=20)
plt.ylabel("log SFR / $M_\star$ (yr$^{-1}$)", fontsize=20)

plt.grid(True)

plt.xlim([7, 12])
plt.ylim([-14, -8])

# plt.savefig("z0mgs_sample_figures/sfrms_all_m31_m33.png", dpi=600, bbox_inches='tight')
# plt.savefig("z0mgs_sample_figures/sfrms_all_m31_m33.pdf", dpi=600, bbox_inches='tight')


Moving out to $D<5$ Mpc includes $\sim50$ galaxies. Most of these galaxies have been observed on $\sim100$s pc scales, approximately the limit of current radio interfometers.

In [ ]:

plt.figure(figsize=(8., 8.))

sb.kdeplot(tab['logM*'], tab['logSFR'] - tab['logM*'])

# 5 Mpc
dist5 = np.logical_and(tab['D'] < 5., tab['logM*'] > 8.)

plt.scatter(tab['logM*'][dist5], (tab['logSFR'] - tab['logM*'])[dist5],
            color=sb.color_palette('colorblind')[1], zorder=1)

print(f"Number of galaxies: {dist5.sum()}")

# M31
plt.scatter(tab['logM*'][tab['D'] < 1][0],
            (tab['logSFR'] - tab['logM*'])[tab['D'] < 1][0],
            color=sb.color_palette('colorblind')[2],
            marker='s', s=160, zorder=10)
# M33
plt.scatter(tab['logM*'][tab['D'] < 1][2],
            (tab['logSFR'] - tab['logM*'])[tab['D'] < 1][2],
            color=sb.color_palette('colorblind')[2],
            marker='s', s=160, zorder=10)

plt.xlabel("log $M_\star$ ($M_\odot$)", fontsize=20)
plt.ylabel("log SFR / $M_\star$ (yr$^{-1}$)", fontsize=20)

plt.grid(True)

plt.xlim([7, 12])
plt.ylim([-14, -8])

# plt.text(7.5, -13.5, f"{dist5.sum()} galaxies within 5 Mpc with $M_\star>10^8$ M$_\odot$", fontsize=18)

print(f"{dist5.sum()} galaxies within 5 Mpc with $M_\star>10^8$ M$_\odot$")

# plt.savefig("z0mgs_sample_figures/sfrms_all_d5.png", dpi=600, bbox_inches='tight')
# plt.savefig("z0mgs_sample_figures/sfrms_all_d5.pdf", dpi=600, bbox_inches='tight')


At $D<10$ Mpc, there is a sample of $\sim150$ galaxies. Resolving 100 pc scales corresponds to $\sim2^{''}$:

In [ ]:

plt.figure(figsize=(8., 8.))

sb.kdeplot(tab['logM*'], tab['logSFR'] - tab['logM*'])

# 5 Mpc
dist10 = np.logical_and(tab['D'] < 10., tab['logM*'] > 8.)

plt.scatter(tab['logM*'][dist10], (tab['logSFR'] - tab['logM*'])[dist10],
            color=sb.color_palette('colorblind')[1], zorder=1)

print(f"Number of galaxies: {dist10.sum()}")

# M31
plt.scatter(tab['logM*'][tab['D'] < 1][0],
            (tab['logSFR'] - tab['logM*'])[tab['D'] < 1][0],
            color=sb.color_palette('colorblind')[2],
            marker='s', s=160, zorder=10)
# M33
plt.scatter(tab['logM*'][tab['D'] < 1][2],
            (tab['logSFR'] - tab['logM*'])[tab['D'] < 1][2],
            color=sb.color_palette('colorblind')[2],
            marker='s', s=160, zorder=10)

plt.xlabel("log $M_\star$ ($M_\odot$)", fontsize=20)
plt.ylabel("log SFR / $M_\star$ (yr$^{-1}$)", fontsize=20)

plt.grid(True)

plt.xlim([7, 12])
plt.ylim([-14, -8])

# plt.text(7.5, -13.5, f"{dist10.sum()} galaxies within 10 Mpc with $M_\star>10^8$ M$_\odot$", fontsize=18)

print(f"{dist10.sum()} galaxies within 10 Mpc with $M_\star>10^8$ M$_\odot$")

# plt.savefig("z0mgs_sample_figures/sfrms_all_d10.png", dpi=600, bbox_inches='tight')
# plt.savefig("z0mgs_sample_figures/sfrms_all_d10.pdf", dpi=600, bbox_inches='tight')


And finally we reach $D<25$ Mpc, corresponding to $0.^{''}8$ to resolve 100 pc scales, which is approximately the smallest spatial scale where SKA1-mid has uniform sensitivity. Within this range there are $\sim2000$ galaxies where SKA1-mid can produce HI observations similar to our current view of the Local Group galaxies. This sample spans the range of galaxy types in the nearby Universe.

In [ ]:

plt.figure(figsize=(8., 8.))

sb.kdeplot(tab['logM*'], tab['logSFR'] - tab['logM*'])

# 25 Mpc
dist25 = np.logical_and(tab['D'] < 25., tab['logM*'] > 8.)

plt.scatter(tab['logM*'][dist25], (tab['logSFR'] - tab['logM*'])[dist25],
            color=sb.color_palette('colorblind')[1], zorder=1)

print(f"Number of galaxies: {dist25.sum()}")

# M31
plt.scatter(tab['logM*'][tab['D'] < 1][0],
            (tab['logSFR'] - tab['logM*'])[tab['D'] < 1][0],
            color=sb.color_palette('colorblind')[2],
            marker='s', s=160, zorder=10)
# M33
plt.scatter(tab['logM*'][tab['D'] < 1][2],
            (tab['logSFR'] - tab['logM*'])[tab['D'] < 1][2],
            color=sb.color_palette('colorblind')[2],
            marker='s', s=160, zorder=10)

plt.xlabel("log $M_\star$ ($M_\odot$)", fontsize=20)
plt.ylabel("log SFR / $M_\star$ (yr$^{-1}$)", fontsize=20)

plt.grid(True)

plt.xlim([7, 12])
plt.ylim([-14, -8])

# plt.text(7.5, -13.5, f"{dist25.sum()} galaxies within 25 Mpc with $M_\star>10^8$ M$_\odot$", fontsize=18)

print(f"{dist25.sum()} galaxies within 25 Mpc with $M_\star>10^8$ M$_\odot$")

# plt.savefig("z0mgs_sample_figures/sfrms_all_d25.png", dpi=600, bbox_inches='tight')
# plt.savefig("z0mgs_sample_figures/sfrms_all_d25.pdf", dpi=600, bbox_inches='tight')


SKA1-mid will enable detailed 21-cm HI surveys of nearby galaxies within $D<25$ Mpc. This sample will be the first to span the star-forming main sequence at resolve ($100$ pc) scales.
A key result of such a survey will be to directly link the HI to the star forming lifecycle (CNM & WNM, molecular cloud formation, HI cloud envelopes, impact of stellar feedback) across the population of star-forming galaxies.
